I found the instructions in the paper, https://academic.oup.com/jncics/article/2/1/pky002/4942295, and this repository to be unclear and incomplete. The intent of this notebook is to clarify that, for myself and others.

This section is a highly modified version of what is in `README_mutation_processing_commands` run with bash and linux, or tecnically my Mac.

Paper's Mutation Data Source, Reformatting, and Filtering

The file “CosmicCLP_MutantExport.tsv,” version 81, was downloaded on July 17, 2017, from the COSMIC cell line project (CLP) online database using the Sanger Institute SSH file transfer protocol (http://cancer.sanger.ac.uk/cell_lines). The mutation data of 1020 cell lines were formatted and filtered as described in the following sections. The code for these operations is available from GitHub (https://github.com/mcjarvis/Mutation-Signatures-Including-APOBEC-in-Cancer-Cell-Lines-JNCI-CS-Supplementary-Scripts). To further facilitate the reproducibility of our analyses, the mutation data from three cell lines are used as examples in this section (BC-3, BT-474, and NALM-6), and all key raw and processed numbers are provided in Supplementary Tables 1–3 (available online).

First off, you will need a copy of `CosmicCLP_MutantExport.tsv`
For the most recent version, currently v91, https://cancer.sanger.ac.uk/cell_lines/download
* Complete mutation data
* Download whole file

For some older versions, select Scripted Download and follow the instructions. Its a bit of a pain. The paper used v81. The columns have changed so you will need to take that into consideration. Unzipped, this file is just under 3GB. In addition, it is protected so I will not be including it in this repository.

This notebook will use v91.

In [ ]:
date

In [ ]:
head -1 CosmicCLP_MutantExport.v91.tsv

In [ ]:
head -1 CosmicCLP_MutantExport.v91.tsv | awk -F"\t" '{print NF}'

In [ ]:
wc -l CosmicCLP_MutantExport.v91.tsv

**Paper's Step 1:** Download, organize, and filter raw mutation data: The fields cell line name (column 5), mutation (column 18), mutation type (column 20), version of the reference genome (column 23), chromosome position of the mutation (column 24), and DNA strand (column 25) were extracted from the “CosmicCLP_MutantExport.tsv” file using the following command:

`awk ′BEGIN{FS="\t"; OFS="\t"}; 0 !∼ /^#/ {print $5, $18, $20, $23, $24, $25}′CosmicCLP_MutantExport.tsv > cosmic_mut.txt`

The paper's command is a bit incorrect.
The broken filter is unnecessary.
And the column numbers have changed since v81 so make the appropriate changes if necessary.

And I'm not sure that `cosmic_mut.txt` is actually needed in the analysis.

In [ ]:
mkdir -p v91

Change 5, 18, 20, 23, 24 and 25 to 5, 20, 22, 25, 26 and 27 for v91

In [ ]:
awk 'BEGIN{FS=OFS="\t"} {print $5, $20, $22, $25, $26, $27 }' \
    CosmicCLP_MutantExport.v91.tsv > v91/Step1.tsv

In [ ]:
head v91/Step1.tsv
# Sample name	Mutation CDS	Mutation Description	GRCh	Mutation genome position	strand

**Paper's Step 2:** Removal of all non-single-base substitution mutations: All mutations that are not single-base substitutions (eg, insertions, deletions, and complex multibase substitutions) were filtered out of the table, leaving single-base substitution mutations annotated as nonsense, missense, or coding silent substitutions. This essential filtering step reduced the number of mutations in BT-474, BC-3, and NALM-6 from 1595 to 1407, 1537 to 1371, and 3291 to 2962, respectively.

In [ ]:
head v91/Step1.tsv | awk '(NR==1 || $3 ~ /Substitution/)'

In [ ]:
cat v91/Step1.tsv | awk 'BEGIN{FS=OFS="\t"}($3 ~ /Substitution/){print $3}' \
| sort | uniq -c

In [ ]:
awk -F"\t" '(NR==1 || $3 ~ /Substitution/) && ($5 != "")' v91/Step1.tsv > v91/Step2.tsv

In [ ]:
head v91/Step2.tsv

In [ ]:
cat v91/Step1.tsv | awk -F"\t" '{print $1}' \
| sort | uniq -c | awk '( $2 ~ /^(BT-474|BC-3|NALM-6)$/ )'

Expecting ... above

* 1537 BC-3
* 1595 BT-474
* 3291 NALM-6

Only one of the initial counts are only off by 1.

These numbers are MUCH bigger in v91 as compared to v81!

Expecting ... below

* 1371 BC-3
* 1407 BT-474
* 2962 NALM-6
  

In [ ]:
cat v91/Step2.tsv | awk -F"\t" '{print $1}' \
| sort | uniq -c | awk '( $2 ~ /^(BT-474|BC-3|NALM-6)$/ )'

**Paper's Step 3:** Additional filtering to remove nonunique chromosomal positions and file reformatting: All nonunique chromosome positions were filtered out of each cell line individually, which ensures that each mutation has only one associated chromosomal position within a cell line. A tab-separated file was created with chromosome number (eg, “chr1”), chromosomal position, reference allele, alternate (mutant) allele, strand of the substitution, and sample (cell line name) as columns. This table was reordered as follows for subsequent analyses: chr1-chr9, chrX, chrY, chr10-chr22, then by ascending chromosomal position, and it was then saved as a text file. This step reduced mutation numbers in BT-474, BC-3, and NALM-6 from 1407 to 1021, 1371 to 963, and 2962 to 2110, respectively.

In [ ]:
awk 'BEGIN{FS=OFS="\t"}(NR>1){ \
split($5,a,":"); chr=a[1]; split(a[2],p,"-"); pos=p[1]; \
mut=substr($2,length($2)-2); split(mut,m,">");\
print chr, pos, m[1], m[2], $6, $1; \
}' v91/Step2.tsv | sort -n | uniq > v91/Step3.tsv

In [ ]:
head v91/Step3.tsv

In [ ]:
tail v91/Step3.tsv

In [ ]:
cat v91/Step3.tsv | awk '( $6 ~ /^(BT-474|BC-3|NALM-6)$/ ){print $6}' | sort | uniq -c 

Expecting ... above

* 963 BC-3
* 1021 BT-474
* 2110 NALM-6

Spot on. Still gotta re-order, the purpose of which I don't understand.

## DO I REALLY NEED TO REORDER HERE?

In [ ]:
wc -l v91/Step3.tsv

**Paper's Step 4:** Filter out “nonmutations” and “nonmatching mutations”: The hg38 reference genome (FastA file, GCA_000001405.2) was used to filter out nonmutations and nonmatching mutations (downloaded from http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/ on July 16, 2017). Nonmutations are instances in which the alternate (mutant) allele matches the reference genome at that position. Nonmatching mutations are instances in which the reference allele does not match the reference genome at that position. These anomalies were filtered out of the single-base substitution mutation data set. This step caused a modest reduction in mutation numbers in a small number of cell lines. For instance, the numbers above for BT-474 and BC-3 were unchanged, but the number for NALM-6 reduced from 2110 to 2108. These single-base substation numbers were used to plot medians in Figure 1, and raw values are listed for each trinucleotide context in Supplementary Table 1 (available online). Following the filtering steps described above, the total single-base substitution mutation count across all cell lines was 663 075.

In [ ]:
head v91/Step3.tsv

In [ ]:
mkdir -p v91/Step4a

In [ ]:
date

Should I be converting to complement for reverse (-) strand? Or was this a bad idea?

I downloaded hg38.fa.gz. I gunzipped it, removed the "chr" prefix, bgzipped and finally indexed it.
```
zcat hg38.fa.gz | sed 's/^>chr/>/' > hg38_num.fa
samtools faidx hg38_num.fa
```

This next step will take quite a while, about 3 hours, with all of the reference checks.

In [ ]:
awk 'BEGIN{FS=OFS="\t";
    comp["A"]="T"
    comp["T"]="A"
    comp["C"]="G"
    comp["G"]="C"
    comp["N"]="N"
}{
    chr=$1; pos=$2; ref=$3; alt=$4; strand=$5; sample=$6
    if(chr==23) chr="X"
    if(chr==24) chr="Y"
    #cmd="samtools faidx hg38.fa chr"chr":"pos"-"pos" | tail -1 "
    cmd="samtools faidx hg38_num.fa "chr":"pos"-"pos" | tail -1 "
    cmd|getline ref38
    close(cmd)
    ref38=toupper(ref38)
    ref=toupper(ref)
    alt=toupper(alt)
    if(strand == "-"){ ref=comp[ref]; alt=comp[alt] }
    if( ref == ref38 ) {
        if( alt == ref38 ){
#Nonmutations are instances in which the alternate (mutant)
#allele matches the reference genome at that position.
            print "Nonmutation?", chr, pos, ref38, ref, alt, strand, sample >> "v91/Step4a-nonmutation.tsv"
            close("v91/Step4a-nonmutation.tsv")
        }else{
            filename=$6
            gsub("/","_",filename)
            filename="v91/Step4a/"filename".tsv"
            print chr, pos, ref, alt, strand, sample >> filename
            close(filename)
        }
    }else{
#Nonmatching mutations are instances in which the reference 
#allele does not match the reference genome at that position.
        print "Nonmatching mutation?", chr, pos, ref38, ref, alt, strand, sample >> "v91/Step4a-nonmatching.tsv"
        close("v91/Step4a-nonmatching.tsv")
    }
}' v91/Step3.tsv

In [ ]:
date

In [ ]:
wc -l v91/Step3.tsv

Expecting Step4 to be only 663075 but won't be.

In [ ]:
cat v91/Step4a/*.tsv | wc -l

In [ ]:
wc -l v91/Step4a-nonmatching.tsv

In [ ]:
# wc -l v91/Step4a-nonmutation.tsv

wc: Step4a-nonmutation.tsv: No such file or directory

In [ ]:
mkdir -p v91/Step4b

In [ ]:
#for f in *-Step4a.tsv ; do n=${f/Step4a/Step4b}; sort -n $f > $n ;  done
for f in v91/Step4a/*tsv; do
    b=$( basename $f ) 
    sort -n $f > v91/Step4b/$b
done

In [ ]:
cat v91/Step4b/*.tsv | wc -l

Combine and remove "strand" column

In [ ]:
cat v91/Step4b/*.tsv | awk 'BEGIN{FS=OFS="\t"}{print $1,$2,$3,$4,$6}' > v91/cosmic_mut.txt

May need to install some perl modules

`cpan install -T XML::DOM::XPath BioPerl Bio::DB::Fasta`

Also, chromosome naming convention must match so you may need to add "chr" prefix to dataset here or in count_trinuc_muts_v8.pl script. The hg38 reference includes the "chr" prefix. hg38_num does not.

In [ ]:
head v91/cosmic_mut.txt

In [ ]:
#for f in Step4b/*.tsv; do
#    ./count_trinuc_muts_v8.pl pvcf hg38.fa $f > $f.count.txt
#done

In [ ]:
#rename 's/tsv.\d*.count/count/' *-Step4b.tsv.*.count.txt
#rename 's/tsv.\d*.count/count/' Step4b/*.tsv.*.count.txt

In [ ]:
#head -1 $( ls Step4b/*count.txt | head -1 ) > cosmic_mut_all_sort.txt
#tail -n +2 -q Step4b/*count.txt >> cosmic_mut_all_sort.txt

In [ ]:
cat v91/Step4b/*.tsv > v91/Step4b.tsv

In [ ]:
wc -l v91/Step4b.tsv

In [ ]:
#awk 'BEGIN{FS=OFS="\t"}{print "chr"$1,$2,$3,$4,$5,$6}' v91/Step4b.tsv > v91/Step4c.tsv
awk 'BEGIN{FS=OFS="\t"}{print $1,$2,$3,$4,$5,$6}' v91/Step4b.tsv > v91/Step4c.tsv

In [ ]:
head v91/Step4c.tsv

In [ ]:
#./count_trinuc_muts_v8.pl pvcf hg38.fa v91/Step4c.tsv > v91/cosmic_mut_all_sort.txt
./count_trinuc_muts_v8.pl pvcf hg38_num.fa v91/Step4c.tsv > v91/cosmic_mut_all_sort.txt

In [ ]:
head v91/cosmic_mut_all_sort.txt

`count_trinuc_muts_v8.pl` filters quite a few mutations out for some reason.

In [ ]:
wc -l v91/cosmic_mut_all_sort.txt

The supplementary data associated with this paper pky002_supp.zip contains
* Supplementary Table 1-mutation counts.xls  
* Supplementary Table 2-COSMIC signatures.xls  
* Supplementary Table 3-APOBEC signatures.xls
  
2 of these spreadsheets include sample and tissue columns which I extracted to create the tab-separated, 2-column cosmic_tissue_type.txt file. There is no header in this file and column 1 is the Cell Line and column 2 is the Tissue of Origin.

In [ ]:
head "Supplementary Table 1-mutation counts.csv"

In [ ]:
awk 'BEGIN{FS=",";OFS="\t"}(NR>1){print $2,$1}' \
    "Supplementary Table 1-mutation counts.csv" > v91/cosmic_tissue_type.txt

In [ ]:
head v91/cosmic_tissue_type.txt

In [ ]:
date

This notebook could be run outside

```
jupyter nbconvert --to notebook --execute Reproduce-1.ipynb --output Reproduce-1-out.ipynb --ExecutePreprocessor.timeout=14400
```